In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs
import time
import random
import re

import psycopg2
import copy

from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from gensim.models.keyedvectors import FastTextKeyedVectors
#fasttext = FastTextKeyedVectors.load("/Users/lilyakhoang/input/araneum_none_fasttextskipgram_300_5_2018/araneum_none_fasttextskipgram_300_5_2018.model")
fasttext = FastTextKeyedVectors.load('/Users/nigula/input/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')

In [2]:
from string import punctuation
full_punctuation = punctuation + "–" + "," + "»" + "«" + "…" +'’'

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [4]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
cursor = conn.cursor()

In [31]:
def get_def_info(word, word_id, from_lang, to_lang, multisence_dict, print_ouput = False):
    multisence_dict[word] = {'word_id': word_id, 'lemma_branch':[], 'wordforms_branch':{}}
    definitions = []
    same_page = False
    for index in range(1,4):
        url = "https://dictionary.cambridge.org/dictionary/" + from_lang + "-" + to_lang + "/" + word + "_" + str(index)
        if print_ouput:print(url)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8' 
        soup = bs(response.text, 'html.parser')
        current_definition = {}
        for link in soup.find_all(["span","b"]):
    
            classes = link.get('class')
            if classes and "def" in classes:
                if link.text.strip() in definitions:
                    same_page = True
                    break
                if print_ouput: print("DEFINITION", link.text.strip())
                current_definition['definition'] = link.text.strip()
                definitions.append(link.text.strip())
                
            if classes and "def-body" in classes:
                if print_ouput:print("RUS_DEFIN", link.find("span", attrs = {'class':"trans"}).text.strip())
                current_definition['local_word'] = link.find("span", attrs = {'class':"trans"}).text.strip()
                current_definition['examples'] = []
                examples_count = 0
                for ex in link.find_all("span", attrs = {'class':"eg"}):
                    if examples_count > 3: break
                    current_definition['examples'].append(ex.text.strip())
                    examples_count += 1
                    if print_ouput:print("ENG_DEFIN_EX", ex.text.strip())
                if print_ouput:print(current_definition)
                multisence_dict[word]['lemma_branch'].append(current_definition)
                current_definition = {}
                if print_ouput:print("===")

        if same_page == True:
            break
        time.sleep(random.uniform(0.01,0.1))
sence_dict = {}      
get_def_info("book", 10, "english","russian",sence_dict)


In [32]:
sence_dict['book']


{'word_id': 10,
 'lemma_branch': [{'definition': 'a set of pages fastened together in a cover for people to read',
   'local_word': 'книга',
   'examples': ['a book about animals']},
  {'definition': 'a set of stamps, tickets, etc that are fastened together inside a cover',
   'local_word': 'книжечка',
   'examples': []},
  {'definition': 'a set of pages fastened together in a cover and used for writing on',
   'local_word': 'записная книжка',
   'examples': ['an address book']},
  {'definition': 'to arrange to use or do something at a particular time in the future',
   'local_word': 'бронировать',
   'examples': ['to book a ticket/hotel room',
    "We've booked a trip to Spain for next month.",
    'Sorry, the hotel is fully booked (= has no more rooms).']},
  {'definition': 'to officially accuse someone of a crime',
   'local_word': 'заводить дело',
   'examples': ['Detectives booked him for resisting arrest.']},
  {'definition': 'If a sports official books you, they write an officia

In [6]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def nltk_lemmatize(word):
    pos = pos_tag(word_tokenize(word))[0][1]
    wordnet_pos = get_wordnet_pos(pos)
    if wordnet_pos:
        lemma = lemmatizer.lemmatize(word, pos = wordnet_pos)
    else:
        lemma = lemmatizer.lemmatize(word)
    return lemma

nltk_lemmatize("having")

'have'

In [28]:
def get_sencewords_and_examples_reverso(eng_wordform, from_lang, to_lang, print_output = False):
    wordform_localsencewords_examples_dict = {}
    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + eng_wordform 
    login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
    header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=header_main, data = login)
    response.encoding = 'utf-8' 
    soup = bs(response.text, 'html.parser')
    first_string_passed = False
    translations_variants = []
    for link in soup.find_all("a", attrs={"class" : "translation"}):
        if first_string_passed == True:
            sence_word = link.text.strip()
            if print_output:print(sence_word)
            wordform_localsencewords_examples_dict[sence_word] = []
            translations_variants.append(sence_word)
        first_string_passed = True
    time.sleep(random.uniform(0.01,0.2))
    for transl_word in translations_variants:
        time.sleep(random.uniform(0.01,0.2))
        url_example = "https://context.reverso.net/перевод/" + to_lang + "-" + from_lang + "/" + transl_word
        response_ex = requests.get(url_example, headers=header_main)
        soup_sub = bs(response_ex.text, 'html.parser')
        if print_output:
            print("========================================")
            print("\nEXAMPLES FOR SENCE ",transl_word,"\n" )
        examples_count = 0
        for l in soup_sub.find_all('span', attrs = {'class':'text', 'lang':'en'}):
            example = l.text.strip()
            #print(example, word in example, type(l.text.strip()), word)
            regex = "\W" + eng_wordform + "\w{0,1}\W"
            find = re.findall(regex, example)
            if len(find) > 0 :
                wordform_localsencewords_examples_dict[transl_word].append(example)
                if print_output: print(example)
                examples_count += 1
                if examples_count >= 3: break
    return wordform_localsencewords_examples_dict
w_locsence_ex_dict = get_sencewords_and_examples_reverso('having',"английский","русский", print_output = True )   
    

наличие
после
имеющих
имеющие
чтобы
имея
весело
необходимости
того
оказывает
есть
имеют
имеет
поскольку
оказывают

EXAMPLES FOR SENCE  наличие 


EXAMPLES FOR SENCE  после 


EXAMPLES FOR SENCE  имеющих 

Such policy completely ensures enrolment in the higher education system of students having such social status.
Article 120 of the Labour Code fixes the duration of basic leave for employees recognized as having rendered special service to Azerbaijan.
Such persons should be coded as having no fixed place of work.

EXAMPLES FOR SENCE  имеющие 

The President was given the authority to issue decrees having the force of law.
The Indian Supreme Court has interpreted such provisions as having real significance.
Grant me, Divine Father, the delegation of having power over malevolent spirits.

EXAMPLES FOR SENCE  чтобы 


EXAMPLES FOR SENCE  имея 

Not having a rope, you can easily lose each other.
It's like I'm having some kind of a breakdown.
They're just having a big meeting about next fal

In [29]:
w_locsence_ex_dict

{'наличие': [],
 'после': [],
 'имеющих': ['Such policy completely ensures enrolment in the higher education system of students having such social status.',
  'Article 120 of the Labour Code fixes the duration of basic leave for employees recognized as having rendered special service to Azerbaijan.',
  'Such persons should be coded as having no fixed place of work.'],
 'имеющие': ['The President was given the authority to issue decrees having the force of law.',
  'The Indian Supreme Court has interpreted such provisions as having real significance.',
  'Grant me, Divine Father, the delegation of having power over malevolent spirits.'],
 'чтобы': [],
 'имея': ['Not having a rope, you can easily lose each other.',
  "It's like I'm having some kind of a breakdown.",
  "They're just having a big meeting about next fall's chorus program."],
 'весело': ['You know, for a moment there, you were actually having fun, Detective.',
  'It looked like you were having so much fun together.'],
 'необ

In [96]:
np.zeros(300,)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [165]:
full_punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~–,»«…’'

In [8]:

l = ['книга', 'книжечка', 'записная книжка', 'бронировать', 'заводить дело', 'штрафовать (в спорте)']
word = 'забронировать'

def look_for_sublemma_word(wordform, lemmas_list, apply_w2v = False):
    clean_wordform = ''
    for ch in wordform:
        if ch not in full_punctuation:
            clean_wordform += ch
    wordform = clean_wordform
    
    if len(wordform.split()) == 1 and apply_w2v == True:
        p = morph.parse(wordform)[0]
        lemma = p.normal_form
        wordform = np.array(fasttext[lemma]).reshape(1, -1)
    elif len(wordform.split()) > 1 and apply_w2v == True:
        try:
            words_vector = np.zeros(300).reshape(1, -1)
            count = 0 
            for word in wordform.split():
                p = morph.parse(word)[0]
                lemma = p.normal_form
                words_vector += np.array(fasttext[lemma]).reshape(1, -1)
                count += 1
            wordform = words_vector / count
        except:
            pass
        
        
    if isinstance(lemmas_list, list) or isinstance(lemmas_list, set):
        for lemma in lemmas_list:
            if len(lemma.split()) == 1:
                if wordform in lemma or lemma in wordform:
                    #print(wordform)
                    return lemma
        return None
    elif isinstance(lemmas_list, str):
        if apply_w2v == False:
            if apply_w2v == False:
                if wordform in lemmas_list or lemmas_list in wordform:
                    return lemmas_list
                else:
                    return None
        
        elif apply_w2v == True:
            words_vector = np.zeros(300).reshape(1, -1)
            try:
                count = 0 
                for word in lemmas_list.split():
                    p = morph.parse(word)[0]
                    lemma = p.normal_form
                    words_vector += np.array(fasttext[lemma]).reshape(1, -1)
                    count += 1
                words_vector = words_vector / count
                return list(cosine_similarity(words_vector, wordform))[0][0]
            except:
                return None
        
    else:
        return None #"no type" + str(type(lemmas_list))

#look_for_sublemma_word("желтая пломба", 'желтая пломба', apply_w2v = True) 
look_for_sublemma_word("занятый член", 'занятый хер выавы', apply_w2v = True) 
            

0.49991900191938105

In [38]:
def merge_lemma_context_vs_oxford_examples(eng_lemma, local_wordsence_examples_dict, general_dict, print_output = False):
    oxford_sensewords_list = []
    oxford_lemmas_examples_pointers_dict = {}
    for definition_element in general_dict[eng_lemma]['lemma_branch']:
        #if print_output: print(definition_element)
        oxford_sensewords_list.append(definition_element['local_word'])
        oxford_lemmas_examples_pointers_dict[definition_element['local_word']] = definition_element['examples']
    if print_output: print("WILL LOOK THROUGH LOCAL WORDFORMS", oxford_sensewords_list)
    oxford_sensewords_list = set(oxford_sensewords_list)
    for context_local_lemma in local_wordsence_examples_dict.keys():
        if print_output: print("\nNOWPROCESSING", context_local_lemma)
        orig_intersection_lemma = look_for_sublemma_word(context_local_lemma, oxford_sensewords_list)
        if not orig_intersection_lemma:
            if print_output: print(context_local_lemma, "\nno such lemma in orig dict")
            pass
        elif len(local_wordsence_examples_dict[context_local_lemma]) == 0:
            if print_output: print(context_local_lemma, "\ndoes not have examples with english wodform")
            pass
        else:
            #general_dict[eng_lemma]['lemma_branch'][context_local_lemma]['examples'].extend(local_wordsence_examples_dict[context_local_lemma][:3])
            if print_output:print(oxford_lemmas_examples_pointers_dict[orig_intersection_lemma])
            examples_len = len(oxford_lemmas_examples_pointers_dict[orig_intersection_lemma])
            extend_len = 4 - examples_len
            if extend_len >0:
                oxford_lemmas_examples_pointers_dict[orig_intersection_lemma].extend(local_wordsence_examples_dict[context_local_lemma][:extend_len])
            if print_output:print("add",local_wordsence_examples_dict[context_local_lemma][:3] )
#dct = copy.deepcopy(sence_dict)
#merge_lemma_context_vs_oxford_examples("book",w_locsence_ex_dict, dct, print_output = True)        
#dct


In [10]:
def merge_locsence_ex_vs_lemma_structure(eng_wordform, eng_lemma,wordform_id, local_wordsence_examples_dict, general_dict, print_output = False):
    general_dict[eng_lemma]['wordforms_branch'][eng_wordform] = {'word_id':wordform_id,'wordsenses':[]}
    oxford_sensewords_list = []
    for definition_element in general_dict[eng_lemma]['lemma_branch']:
        #if print_output: print(definition_element)
        oxford_sensewords_list.append(definition_element['local_word'])
    if print_output: print("WILL LOOK THROUGH LOCAL WORDFORMS", oxford_sensewords_list)
    oxford_sensewords_list = set(oxford_sensewords_list)
    successfull_merge_count = 0
    for local_wordform in local_wordsence_examples_dict.keys():
        p = morph.parse(local_wordform)[0]
        local_lemma = p.normal_form
        if print_output: print("\nNOWPROCESSING", local_wordform, local_lemma, look_for_sublemma_word (local_lemma, oxford_sensewords_list))
        if look_for_sublemma_word (local_wordform, oxford_sensewords_list):
            if print_output: print(local_wordform, "\nis original lemma itself")
            pass
        elif len(local_wordsence_examples_dict[local_wordform]) == 0:
            if print_output: print(local_wordform, "\ndoes not have examples with english wodform")
            pass
        else:
            if print_output: print(local_wordform, "\nno match but no limitations")
            local_wordform_dict = {"local_word":local_wordform,"examples":local_wordsence_examples_dict[local_wordform][:3]}
            if print_output: print("LOCAL_DICT_OBJECT",local_wordform_dict)
            general_dict[eng_lemma]['wordforms_branch'][eng_wordform]['wordsenses'].append(local_wordform_dict)
        """
        elif look_for_sublemma_word (local_lemma, oxford_sensewords_list) == True:
            if print_output: print(local_wordform, "\nlocal_wordform to local lemma or viceversa match")
            successfull_merge_count += 1
            local_wordform_dict = {"sence_wordform_local":local_wordform,
                                   "context":[]}
            for definition_element in general_dict[eng_lemma]['lemma_branch']:
                
                if look_for_sublemma_word (local_lemma, definition_element['local_word']):
                    local_wordform_examples = copy.deepcopy(definition_element['examples'])
                    local_wordform_dict['context'] = local_wordform_examples
                    try:
                        local_wordform_dict['context'].extend(local_wordsence_examples_dict[local_wordform][:3])
                        definition_element['examples'].extend(local_wordsence_examples_dict[local_wordform][:3])
                    except:
                        local_wordform_dict['context'].extend(local_wordsence_examples_dict[definition_element['local_word']][:3])
                        definition_element['examples'].extend(local_wordsence_examples_dict[definition_element['local_word']][:3])
            general_dict[eng_lemma]['wordforms_branch'][eng_wordform].append(local_wordform_dict)
            if print_output: print("LOCAL_DICT_OBJECT",local_wordform_dict)
        """
    if print_output:
        if successfull_merge_count == 0: print("NO LOCAL INTERSECTIONS FOR WORDWORM", eng_wordform)
    if len(general_dict[eng_lemma]['wordforms_branch'][eng_wordform]['wordsenses']) == 0:
        del general_dict[eng_lemma]['wordforms_branch'][eng_wordform]
        
        
#merge_locsence_ex_vs_lemma_structure("booking","book",w_locsence_ex_dict, big_diCKt, print_output = True)        
#big_diCKt

In [ ]:
time.now()

In [5]:
import json
with open("one_word_transalte.json", "r") as f:
    one_word_transalte = json.load(f)

In [109]:
list(cosine_similarity((np.array(fasttext["мама"]).reshape(1, -1) + np.array(fasttext["папа"]).reshape(1, -1) / 2), np.array(fasttext["мамка"]).reshape(1, -1)))[0][0]

0.6401495

In [147]:
a = {'a':1}
del a['a']
print(a)

{}


In [20]:
def ll_translate(text):
    url_y = "http://192.168.122.15:30556/getTranslates"
    h_y = {"apiVersion": "1.0.0", "text": text, "langPair": {"source": "en", "target": "ru"}}
    #h_y = {"apiVersion": "1.0.0", "text": "Landing to the mars", "langPair": "en-ru"}
    r = requests.post(url = url_y, json=h_y)
    data = r.json()
    return data['translate'][0]['translate_value']

ll_translate("mother")

'мать'

In [47]:
def yandex_translate(text, from_lang, to_lang):
    url_y = "http://192.168.122.13:31436/GetTranslate"
    h_y = {"apiVersion": "1.0.0", "text": text, "langPair": {"source": from_lang, "target": to_lang}}
    r = requests.post(url = url_y, json=h_y)
    data = r.json()
    if 'translate' in data:
        return data['translate']
    else:
        return None
yandex_translate("привет уеба", "ru", "fr")


'salut, connard.'

In [ ]:
h_y = {"apiVersion": "1.0.0", "text": wordsenses['definition'], "langPair": {"source": "en", "target": "ru"}}
wordsenses['definition_local'] = requests.post(url = url_y, json=h_y).json()['translate'][0]['translate_value']

In [ ]:
def get_most_similar_lldb_instance(new_senseword_json, lldb_instances_dict, used_words_from_lldb, debug = False):
    similarity_dict = {}
    
    for sense_word_lldb in lldb_instances_dict.keys():
        
        if sense_word_lldb not in used_words_from_lldb:
            if debug == True: print(new_senseword_json['local_word'], sense_word_lldb)
            similarity_dict[sense_word_lldb] = look_for_sublemma_word(new_senseword_json['local_word'],sense_word_lldb, apply_w2v = True)

    for sorted_lldb_element in sorted(similarity_dict.items(), key=lambda kv: kv[1], reverse = True):
        if sorted_lldb_element[1] < 0.6:
            if debug == True:print("run out of similar ex")
            new_senseword_json['word_rating'] = 0
            return None
        elif sorted_lldb_element[1] >= 0.6 and sorted_lldb_element[0] not in used_words_from_lldb:
            if debug == True:print("word_rating similarity from",sorted_lldb_element[0] )
            new_senseword_json['word_rating'] = lldb_instances_dict[sorted_lldb_element[0]]['word_rating']
            return sorted_lldb_element[0]
    if debug == True: print("no similarities")
    new_senseword_json['word_rating'] = 0
    return None
    
        
def one_word_processing(wordsence_example_dict,word,  debug = False):
    #word_id + wordsences  for wordform
    #word_id + lemma_branch for lemma
    if 'lemma_branch' in wordsence_example_dict:
        if debug: print( "\n\nLEMMA_EXAMPLES_ARE_BEING_PROCESSED", word)
        wordsenses_list = wordsence_example_dict['lemma_branch']
    elif 'wordsenses' in wordsence_example_dict:
        if debug: print("\n\nWORDFORM_EXAMPLES_ARE_BEING_PROCESSED", word)
        wordsenses_list = wordsence_example_dict['wordsenses']
    request = """SELECT 
    content_words_lang.word_rating,
    content_words_lang.jdesc->>'tr' AS translate,
    content_words_lang.jdesc->>'ctx' AS context

    FROM content_words_lang
    where content_words_lang.jdesc->>'ctx' is not null and content_words_lang.ref_id = 1 and 
    content_words_lang.word_id = """+ str(wordsence_example_dict['word_id']) +""" and content_words_lang.word_rating > 0
    LIMIT 100 """
    cursor.execute(request)
    lldb_examples_dict = {}
    for wordform in cursor:
        lldb_examples_dict[wordform[1]] = {"word_rating":wordform[0],"eng_example":wordform[2]}
    if debug == True: print("lldb_examples_dict", lldb_examples_dict)
    
    used_lldb_words = []
    for wordsenses in wordsenses_list:
        #translate definition and examples
        #print(wordsenses)
        """
        if 'definition' in wordsenses:
            try:
                wordsenses['definition_local'] = ll_translate(wordsenses['definition'])
            except:
                wordsenses['definition_local'] = yandex_translate(wordsenses['definition'], "en", "ru")
                if not wordsenses['definition_local']:
                    print("FAILED TO TRANSLATE",wordsenses['definition'])
        
        translated_examples_list = []
        for example in wordsenses['examples']:
            try:
                translated_example = ll_translate(example)
            except:
                translated_example = yandex_translate(example, "en", "ru")
            translated_examples_list.append(translated_example)
        wordsenses['examples_local'] = translated_examples_list
        """
        if len(list(lldb_examples_dict.keys())) == len(used_lldb_words):
            if debug == True:print("run of of lldb words")
            wordsenses['word_rating'] = 0
        if debug == True: 
            print(wordsence_example_dict['word_id'], wordsenses)
            print()
        used_word = get_most_similar_lldb_instance(wordsenses, lldb_examples_dict, used_lldb_words)
        if used_word:
            used_lldb_words.append(used_word)
            if debug == True:print("used lldf words", used_lldb_words)
            
def get_similar_merge_pictures_rating_and_translate(wodrs_dict, debug = False):
    for eng_lemma in wodrs_dict.keys():
        if debug == True:print(eng_lemma)
        one_word_processing(wodrs_dict[eng_lemma],eng_lemma, debug = False )
        for wordform in wodrs_dict[eng_lemma]['wordforms_branch'].keys():
            one_word_processing(wodrs_dict[eng_lemma]['wordforms_branch'][wordform],wordform )
        
#copy_big_diCKt = copy.deepcopy(big_diCKt)      
#get_similar_merge_pictures_rating(copy_big_diCKt)       

In [33]:
copy_big_diCKt

NameError: name 'copy_big_diCKt' is not defined

In [39]:
def process_lemma_and_forms(word, dictionary, show_words = False):
    req = """SELECT DISTINCT
    content_words.word_id,content_words.word_lemma, content_words.word_value
    FROM content_words
    WHERE content_words.word_value !~ ('\W') AND
    (array_length(regexp_split_to_array(content_words.word_value, '[ ''-]'), 1) = 1) 
    and word_lemma != 0 and word_hash = calc_hash(' """ + word + """')"""
    cursor.execute(req)

    
    req_res = cursor.fetchone()
    if not req_res:
        if show_words == True: print(word, " is not in db")
        return 0
    else:
        #print(a[0])
        word_id = req_res[0]
        if show_words == True: print(req_res[2])
            
    #moment_1 = time.time()
    get_def_info(word, word_id, "english","russian",dictionary)
    #moment_2 = time.time()
    #print("get_def_info",moment_2 - moment_1)
    
    lemma_sence_ex_dict = get_sencewords_and_examples_reverso(word,"английский","русский", print_output = False )
    #moment_3 = time.time()
    #print("get_sencewords_and_examples_reverso",moment_3 - moment_2)
    
    merge_lemma_context_vs_oxford_examples(word,lemma_sence_ex_dict, dictionary, print_output = False) 
    #moment_4 = time.time()
    #print("merge_lemma_context_vs_oxford_examples",moment_4 - moment_3)
    
    word_forms_request = """SELECT DISTINCT 
    content_words.word_value,
    content_words.word_id
    FROM content_words
    where word_lemma = """ + str(word_id) + """  and word_lemma != word_id """
    cursor.execute(word_forms_request)
    #moment_6 = time.time()
    #print("word forms request to lldb",moment_6 - moment_5)
    for wordform in cursor:
        if show_words == True: print(wordform[0])
        wordform_lemma = nltk_lemmatize(wordform[0])
        if wordform_lemma == word:
            if show_words == True: print("will process", wordform[0])
            local_wordforms_sence_examples_dict = get_sencewords_and_examples_reverso(wordform[0],"английский", "русский")  
            merge_locsence_ex_vs_lemma_structure(wordform[0],word, wordform[1], local_wordforms_sence_examples_dict,dictionary ,
                                                                                 print_output= False)
            #если после этого узел englsih_wordform оказывается пустым списком значит reverso не нашел нормальной словесной единицы для этого
            #то есть слово скорее всгео херовое как показала практика типа haveing haved
        else:
            if show_words == True: print("will not process", wordform[0], "lemma is",wordform_lemma )
        #get_def_info(wordform[0], "english","russian",dictionary,is_word_form = True, original_word = word )
    get_similar_merge_pictures_rating_and_translate(dictionary)
        
big_diCKt = {}        
process_lemma_and_forms("book",big_diCKt)

In [ ]:
lemma_branch >> arrange to use or do something at a particu

In [40]:
big_diCKt

{'book': {'word_id': 5756,
  'lemma_branch': [{'definition': 'a set of pages fastened together in a cover for people to read',
    'local_word': 'книга',
    'examples': ['a book about animals',
     'Recently has published as a co-author with Ra-Ma the book entitled Internet protocols.',
     'The book aimed to sensitize children to the different aspects of poverty.',
     'The book is available on the OHCHR web site.'],
    'definition_local': 'набор страниц, скрепленных вместе в обложке для людей, чтобы читать',
    'examples_local': ['книга о животных',
     'Недавно опубликовал в соавторстве с РА-Ма книгу под названием интернет-протоколы.',
     'Цель этой книги-привлечь внимание детей к различным аспектам нищеты.',
     'С этой книгой можно ознакомиться на вебсайте УВКПЧ.'],
    'word_rating': 163702},
   {'definition': 'a set of stamps, tickets, etc that are fastened together inside a cover',
    'local_word': 'книжечка',
    'examples': ['Have you got an autograph book?',
     

In [ ]:
import json
with open("one_word_transalte.json", "w") as f:
    json.dump(big_diCKt, f, ensure_ascii=False, indent = 4)

In [ ]:
big_diCKt

In [16]:
import pandas as pd
from tqdm import tqdm

In [17]:
test_words_db = pd.read_csv("test_words.csv")
test_words = list(test_words_db['word'])

In [ ]:

  0%|          | 0/200 [00:00<?, ?it/s]


  0%|          | 1/200 [00:08<28:14,  8.52s/it]


  1%|          | 2/200 [00:22<33:04, 10.02s/it]


  2%|▏         | 3/200 [01:02<1:02:31, 19.04s/it]


In [50]:
global_dict = {}
count = 0
for word in tqdm(test_words[77:]):
    count += 1
    process_lemma_and_forms(word.lower(), global_dict)
    if count % 5 == 0:
        with open("big_dict_with_almost_all_features_v4.json", "w") as f:
            json.dump(global_dict, f, ensure_ascii=False, indent = 4)







  0%|          | 0/123 [00:00<?, ?it/s]





  1%|          | 1/123 [01:18<2:39:29, 78.44s/it]





  2%|▏         | 2/123 [02:02<2:17:32, 68.20s/it]





  2%|▏         | 3/123 [02:18<1:45:07, 52.56s/it]





  3%|▎         | 4/123 [02:38<1:24:48, 42.76s/it]





  4%|▍         | 5/123 [02:59<1:11:12, 36.21s/it]





  5%|▍         | 6/123 [03:15<58:48, 30.16s/it]  





  6%|▌         | 7/123 [03:39<54:40, 28.28s/it]





  7%|▋         | 8/123 [04:08<54:39, 28.52s/it]





  7%|▋         | 9/123 [04:28<49:18, 25.95s/it]





  8%|▊         | 10/123 [04:50<46:43, 24.81s/it]





  9%|▉         | 11/123 [05:13<45:20, 24.29s/it]





 10%|▉         | 12/123 [05:41<47:01, 25.42s/it]





 11%|█         | 13/123 [06:07<46:48, 25.53s/it]





 11%|█▏        | 14/123 [06:42<51:22, 28.28s/it]





 12%|█▏        | 15/123 [07:02<46:24, 25.78s/it]





 13%|█▎        | 16/123 [07:26<45:15, 25.38s/it]





 14%|█▍        | 17/123 [07:56<46:57, 26.58s/it]





 15%|█▍        | 18/123 [08:

KeyboardInterrupt: 